Aim: Create a representation of a DNA construction
    . Create outline
    . Add features
    . Associate info to features

To test the code in the future, it will be used an example that can be found on https://github.com/BjornFJohansson/pydna-examples/blob/master/assembly/assembly.ipynb

In [3]:
from pydna import *

In [5]:
b  = Dseqrecord("agctactgactattaggggttattctgatcatctgatctactatctgactgtactgatcta")
l  = Dseqrecord("AGGCATCTACTG")
c  = Dseqrecord("tctgatctactatctgactgtactgatctattgacactgtgatcattctagtgtattactc")
(b+l+c).looped().cseguid()



frags = assembly_fragments( (primer_design(b), l, primer_design(c), primer_design(b)), 1)

b2 = pcr(frags[-1].forward_primer, frags[0].reverse_primer, b)

asm2 = Assembly( (b2, frags[1], frags[0], frags[0], frags[2]), limit=1, only_terminal_overlaps=True)

asm2.circular_products[0]

Contig(o456)

In [6]:
class Product_repr(object):

    def __init__(self, product, width = 1000):
        self.prod = product
        self.frag_list = product.source_fragments
        self.frag_N = product.number_of_fragments
        self.fragment_width = 150
        self.fragment_heigth = 40
        self.overlap_width = 30
        self.width = width
        self.heigth = 300 + 300*(int(self.frag_N-1)/10)
        
    def _repr_html_(self):
        return "&#x25CB; (<b>html</b>)"
    
    def _repr_javascript_(self):
        from IPython.display import display_html
        html='''<html>
                  <head></head>
                  <body>
                    <canvas id='Prod'></canvas>
                  </body>
               </html>'''
        display_html(html, raw=True)
        
        return self.get_js()
    
    def get_js(self):
        begin_x = 60
        middle_y = 70
        delta_y = self.fragment_heigth + 10
        begin_y = middle_y - delta_y
        end_x = begin_x + self.fragment_width
        end_y_in = begin_y + self.fragment_heigth
        end_y_out = begin_y + self.fragment_heigth/2
        corner = 15
        js = '''
        var c = document.getElementById('Prod');
        var ctx = c.getContext('2d');
        ctx.canvas.width  = {Can_W};
        ctx.canvas.height = {Can_H};
        
        ctx.strokeStyle = "red";
        ctx.font="15px Arial";
        ctx.textAlign="center";
        
        ctx.lineWidth = {C};
        ctx.lineJoin="round";
        
        ctx.beginPath();
        ctx.fillStyle = "yellow";
        ctx.fillRect({Start}, {Begin_y}, {L}, {H});
        ctx.fillStyle = "black";
        ctx.fillText("{Name}", {Start}+({L}/2), {Begin_y}+{H}/2+5);
        
        ctx.beginPath();
        ctx.fillStyle = "red";
        ctx.strokeRect({End}+{C}/2, {Begin_y}+{C}/2, {Ovr}-{C}, {H}-{C});
        ctx.fillRect({End}, {Begin_y}, {Ovr}-{C}, {H});
        ctx.fillStyle = "black";
        ctx.fillText("{N_Ovr}", {End} + {Ovr}/2, {Begin_y}+{H}/2+5);
        '''.format(Can_W = self.width,
                   Can_H = self.heigth,
                   H = self.fragment_heigth,
                   L = self.fragment_width,
                   Start = begin_x,
                   Begin_y = begin_y,
                   End = end_x,
                   Ovr = self.overlap_width,
                   C = corner,
                   Name = self.frag_list[0].name,
                   N_Ovr = self.frag_list[0].right_overlap_size)
        
        for Frag in range(1, self.frag_N):
            orientation = (-1)**int(Frag/5)
            
            if (Frag)%5 == 0 and Frag != self.frag_N:
                middle_y = middle_y + delta_y*4
                start_x = end_x
            else:
                start_x = end_x + orientation*self.overlap_width
            
            end_x = start_x + orientation*self.fragment_width
            new_js='''            
            ctx.beginPath();
            ctx.fillStyle = "red";
            ctx.strokeRect({Start}-({Or}*({Ovr}-{C}/2)), {Mid_y}-(Math.pow(-1,{n})*{Delta})+{C}/2, {Or}*({Ovr}-{C}), {H}-{C});
            ctx.fillRect({Start}-({Or}*({Ovr}-{C}/2)), {Mid_y}-(Math.pow(-1,{n})*{Delta}), ({Or}*({Ovr}-{C}/2)), {H});
            ctx.fillStyle = "black";
            ctx.fillText("{P_Ovr}", {Start} - ({Or}*{Ovr})/2, {Mid_y} -(Math.pow(-1,{n})*{Delta})+{H}/2+5);
            
            ctx.beginPath();
            ctx.lineWidth = 0.5;
            ctx.setLineDash([5, 6]);
            ctx.moveTo({Start} -({Or}*{Ovr}), {y_in});
            ctx.lineTo({Start} -({Or}*{Ovr}), {H} + {Mid_y} -(Math.pow(-1,{n})*{Delta}+{Par}*({H}/2)));
            ctx.stroke();
            ctx.moveTo({Start}, {y_out});
            ctx.lineTo({Start}, {Mid_y} -(Math.pow(-1,{n})*({Delta}+{Par}*{H})));
            ctx.stroke();
            
            ctx.lineWidth = {C};
            ctx.setLineDash([]);
            
            ctx.beginPath();
            ctx.fillStyle = "yellow";
            ctx.fillRect({Start}, {Mid_y} -(Math.pow(-1,{n})*{Delta}), {L}*{Or}, {H});
            ctx.fillStyle = "black";
            ctx.fillText("{Name}", {Start} + {Or}*({L}/2), {Mid_y} -(Math.pow(-1,{n})*{Delta}) +{H}/2+5);
            
            ctx.beginPath();
            ctx.fillStyle = "red";
            ctx.strokeRect({End}+({Or}*({C}/2)), {Mid_y} -(Math.pow(-1,{n})*{Delta})+{C}/2, {Or}*({Ovr}-{C}), {H}-{C});
            ctx.fillRect({End}, {Mid_y} -(Math.pow(-1,{n})*{Delta}), {Or}*({Ovr}-{C}), {H});
            ctx.fillStyle = "black";
            ctx.fillText("{N_Ovr}", {End} + ({Or}*{Ovr})/2, {Mid_y} -(Math.pow(-1,{n})*{Delta})+{H}/2+5);
            '''.format(H = self.fragment_heigth,
                       L = self.fragment_width,
                       Start = start_x,
                       End = end_x,
                       Or = orientation,
                       Mid_y = middle_y,
                       Delta = delta_y,
                       y_in = end_y_in,
                       y_out = end_y_out,
                       P_Ovr = self.frag_list[Frag].left_overlap_size,
                       Ovr = self.overlap_width,
                       Par = Frag%2,
                       C = corner,
                       N_Ovr = self.frag_list[Frag].right_overlap_size,
                       n = Frag,
                       Name = self.frag_list[Frag].name)
            
            if Frag%2 == 1:
                end_y_in = middle_y - (((-1)**Frag)*delta_y)
                end_y_out = middle_y - (((-1)**Frag)*(delta_y))+self.fragment_heigth/2
                
            elif Frag%2 == 0:
                end_y_in = middle_y - (((-1)**Frag)*(delta_y))+self.fragment_heigth
                end_y_out = middle_y - (((-1)**Frag)*(delta_y))+self.fragment_heigth/2
            
            js = js + new_js
            
        if not self.prod.linear and orientation > 0:
            start_x = end_x + orientation*self.overlap_width
            js = js + '''
            ctx.lineWidth = {C}/2
            ctx.beginPath();
            ctx.moveTo({Start}, {Mid_y} -(Math.pow(-1,{n})*{Delta})+({H}/2));
            ctx.lineTo({Start} + 20, {Mid_y} -(Math.pow(-1,{n})*{Delta})+({H}/2));
            ctx.lineTo({Start} + 20, {Mid_y} -(Math.pow(-1,{n})*{Delta}) + 250);
            ctx.lineTo({Begin_x}-({Ovr}+20), {Mid_y} - (Math.pow(-1,{n})*{Delta}) + 250);
            ctx.lineTo({Begin_x}-({Ovr}+20), {Begin_y}+({H}/2));
            ctx.lineTo({Begin_x}-{Ovr}, {Begin_y}+({H}/2));
            ctx.strokeStyle = "blue";
            ctx.stroke();
            
            ctx.lineWidth = {C};
            ctx.strokeStyle = "red"
            ctx.beginPath();
            ctx.fillStyle = "red";
            ctx.strokeRect({Begin_x}-({Ovr}-{C}/2), {Begin_y}+{C}/2, {Ovr}-{C}, {H}-{C});
            ctx.fillRect({Begin_x}-({Ovr}-{C}/2), {Begin_y}, {Ovr}-{C}/2, {H});
            ctx.fillStyle = "black";
            ctx.fillText("{P_Ovr}", {Begin_x}-{Ovr}/2, {Begin_y}+{H}/2+5);
            '''.format(H = self.fragment_heigth,
                       Start = start_x,
                       Mid_y = middle_y,
                       P_Ovr = self.frag_list[0].left_overlap_size,
                       Ovr = self.overlap_width,
                       Begin_y = begin_y,
                       Delta = delta_y,
                       L = self.fragment_heigth,
                       n = Frag,
                       Begin_x = begin_x,
                       C = corner)
        
        elif not self.prod.linear and orientation < 0:
            start_x = end_x + orientation*self.overlap_width
            js = js + '''
            ctx.lineWidth = {C}/2
            ctx.beginPath();
            ctx.moveTo({Start}, {Mid_y} -(Math.pow(-1,{n})*{Delta})+({H}/2));
            ctx.lineTo({Begin_x}-({Ovr}+20), {Mid_y} - (Math.pow(-1,{n})*{Delta})+({H}/2));
            ctx.lineTo({Begin_x}-({Ovr}+20), {Begin_y}+({H}/2));
            ctx.lineTo({Begin_x}-{Ovr}, {Begin_y}+({H}/2));
            ctx.strokeStyle = "blue";
            ctx.stroke();
            
            ctx.lineWidth = {C};
            ctx.strokeStyle = "red"
            ctx.beginPath();
            ctx.fillStyle = "red";
            ctx.strokeRect({Begin_x}-({Ovr}-{C}/2), {Begin_y}+{C}/2, {Ovr}-{C}, {H}-{C});
            ctx.fillRect({Begin_x}-({Ovr}-{C}/2), {Begin_y}, {Ovr}-{C}/2, {H});
            ctx.fillStyle = "black";
            ctx.fillText("{P_Ovr}", {Begin_x}-{Ovr}/2, {Begin_y}+{H}/2+5);
            '''.format(H = self.fragment_heigth,
                       Start = start_x,
                       Mid_y = middle_y,
                       P_Ovr = self.frag_list[0].left_overlap_size,
                       Ovr = self.overlap_width,
                       Begin_y = begin_y,
                       Delta = delta_y,
                       L = self.fragment_heigth,
                       n = Frag,
                       Begin_x = begin_x,
                       C = corner)
            
        else:
             js = js + '''
             ctx.clearRect({End}, {Mid_y} -(Math.pow(-1,{n})*{Delta}), 
             {Or}*{Ovr}, {H});
             '''.format(H = self.fragment_heigth,
                        End = end_x,
                        Mid_y = middle_y,
                        n = Frag,
                        Delta = delta_y,
                        Or = orientation,
                        Ovr = self.overlap_width)
            
        return js

In [7]:
D = Product_repr(asm2.circular_products[0])
D

Notes!!!

In [ ]:
js="""function getMousePos(canvas, event) {
	var rect = canvas.getBoundingClientRect();
	return {
		x: event.clientX - rect.left,
		y: event.clientY - rect.top
	};
}
function isInside(pos, rect){
	return pos.x > rect.x && pos.x < rect.x+rect.width && pos.y < rect.y+rect.heigth && pos.y > rect.y
}

var canvas = document.getElementById('myCanvas');
var context = canvas.getContext('2d');
var rect = {
	x:250,
	y:350,
	width:200,
	heigth:100
};

canvas.addEventListener('click', function(evt) {
	var mousePos = getMousePos(canvas, evt);
    debugger;
	if (isInside(mousePos,rect)) {
		alert('clicked inside rect');
    }else{
        alert('clicked outside rect');
    }	
}, false);

context.beginPath();
context.rect(250, 350, 200, 100); 
context.fillStyle = '#FFFFFF'; 
context.fillStyle = 'rgba(225,225,225,0.5)';
context.fillRect(25,72,32,32);
context.fill(); 
context.lineWidth = 2;
context.strokeStyle = '#000000'; 
context.stroke();
context.closePath();
context.font = '40pt Kremlin Pro Web';
context.fillStyle = '#000000';
context.fillText('Start', 345, 415);"""